In [1]:
import pandas as pd
data = pd.read_csv('LC_15_18_new.csv')

C:\Users\Claire\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (13,37,94,105,106,107,110,111,112,115) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
data.shape

(927931, 129)

### Columns that should be discarded
- 100% missing values: `id`, `member_id`, `url`, `next_pymnt_d`
- The same for all rows: `pymnt_plan`, `out_prncp`, `out_prncp_inv`, and `policy_code`
- Updated in past 2 months -- wouldn't have these values at loan origination: `num_tl_120dpd_2m`, and `num_tl_30dpd`
- Hardship-related variables, relating to borrowers who were on a hardship plan -- wouldn't know at the point of loan origination whether or not the borrower will have a hardship: `deferral_term`, `hardship_amount`, `hardship_dpd`, `hardship_end_date`, `hardship_flag`, `hardship_last_payment_amount`, `hardship_length`, `hardship_loan_status`, `hardship_payoff_balance_amount`, `hardship_reason`, `hardship_start_date`, `hardship_status`, `hardship_type`, `orig_projected_additional_accrued_interest`, `payment_plan_start_date`
- String variables provided by borrowers. LendingClub categorized them into other variables: `desc`, `title`, and `emp_title`
- Loan settlement/charged off related variables -- wouldn't know them at the time of funding: `debt_settlement_flag`, `debt_settlement_flag_date`, `settlement_amount`, `settlement_date`, `settlement_percentage`, `settlement_status`, `settlement_term`, `recoveries`, `collection_recovery_fee`
- Loan payment related variables -- wouldn't know them at the time of funding: `funded_amnt`, `funded_amnt_inv`, `last_pymnt_amnt`, `out_prncp`, `out_prncp_inv`, `total_pymnt`, `total_pymnt_inv`, `total_rec_int`, `total_rec_late_fee`, `total_rec_prncp`, `last_pymnt_d`, `last_credit_pull_d`, `disbursement_method`
- `initial_list_status`: as the values "W' and "F" are randomly assigned
- Columns relating to co-borrowers. These columns only have non-null values from issue dates of 2017-03-01 and later (or 2015-10-01 and later for 'dti_joint', 'annual_inc_joint', and 'verification_status_joint'), and have around 97% missing values, so we remove these columns: `annual_inc_joint`, `dti_joint`, `revol_bal_joint`, `sec_app_chargeoff_within_12_mths`, `sec_app_collections_12_mths_ex_med`, `sec_app_earliest_cr_line`, `sec_app_inq_last_6mths`, `sec_app_mort_acc`, `sec_app_mths_since_last_major_derog`, `sec_app_num_rev_accts`, `sec_app_open_acc`, `sec_app_open_act_il`, `sec_app_revol_util`, `verification_status_joint`, `sec_app_fico_range_low`, `sec_app_fico_range_high`
- Other: `zip_code`

In [35]:
data['application_type'].describe()

count         927931
unique             2
top       Individual
freq          898347
Name: application_type, dtype: object

In [38]:
data['sec_app_fico_range_low'].isnull().sum() * 100 / len(data)

97.6192195324868